In [1]:
import qcodes as qc
from cappy.config import testconfig

station = qc.Station(config_file=str(testconfig))

key = station.load_instrument("keithley")
yoko = station.load_instrument("yoko")
li = station.load_instrument("lockin")

TypeError: 'module' object is not callable